เคส

# 0. Magic Commands


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [2]:
from fastai import *
from fastai.basics import *
from fastai.tabular import *

# 2. ข้อมูล

เคสนี้เราจะใช้ Dataset ตาราง

Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.


You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

Files
train.csv - historical data including Sales
test.csv - historical data excluding Sales
sample_submission.csv - a sample submission file in the correct format
store.csv - supplemental information about the stores
Data fields
Most of the fields are self-explanatory. The following are descriptions for those that aren't.

Id - an Id that represents a (Store, Date) duple within the test set
Store - a unique Id for each store
Sales - the turnover for any given day (this is what you are predicting)
Customers - the number of customers on a given day
Open - an indicator for whether the store was open: 0 = closed, 1 = open
StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
StoreType - differentiates between 4 different store models: a, b, c, d
Assortment - describes an assortment level: a = basic, b = extra, c = extended
CompetitionDistance - distance in meters to the nearest competitor store
CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
Promo - indicates whether a store is running a promo on that day
Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


https://www.kaggle.com/c/rossmann-store-sales

เราจะใช้ข้อมูลภายนอกมาประกอบด้วย

http://files.fast.ai/part2/lesson14/rossmann.tgz

In [3]:
path=Config().data_path()/Path('rossmann/')
table_names = ['train', 'store', 'store_states', 'state_names', 'googletrend', 'weather', 'test']
tables = [pd.read_csv(path/f'{fname}.csv', low_memory=False) for fname in table_names]
train, store, store_states, state_names, googletrend, weather, test = tables
len(train),len(test)

(1017209, 41088)

ls ดูว่ามีไฟล์อะไรบ้าง

In [4]:
path.ls()

[PosixPath('/home/jupyter/.fastai/data/rossmann/state_names.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/test.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/sample_submission.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/googletrend.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/weather.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/store.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/train.csv'),
 PosixPath('/home/jupyter/.fastai/data/rossmann/store_states.csv')]

โหลดไฟล์ adult.csv ขึ้นมาดู

In [5]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


เปิดดู 5 แถวแรก

In [6]:
dataframe.head()

NameError: name 'dataframe' is not defined

ดูสถิติของข้อมูล สังเกต

1. count แต่ละ Column ไม่เท่ากัน แสดงว่ามีข้อมูลขาดไป
2. unique คือ จำนวนหมวดหมู่ของข้อมูลแบบ Category แต่ละ Column
3. top, freq คือ ค่าและจำนวนครั้ง ของข้อมูลที่ปรากฎบ่อยที่สุด
4. mean, std, min, ... max ข้อมูลสถิติ ของข้อมูลตัวเลข

In [ ]:
dataframe.describe(include='all')

# 3. เตรียมข้อมูล



เราต้องบอก Databunch ว่า Column ไหนเป็นข้อมูลชนิดอะไร

1. dependent_variable คือ ข้อมูลที่เราจะทำนาย
2. category_column_names คือ Column ข้อมูลแบบหมวดหมู่ ไม่ต่อเนื่อง มีจำนวนเป็นไปได้จำกัด
3. continuous_column_names คือ Column ข้อมูลตัวเลข แบบค่าต่อเนื่อง มีจำนวนเป็นไปได้ไม่จำกัด
4. preprocesses คือ เราจะเตรียมข้อมูลอย่างไร ก่อนป้อนให้โมเดล เช่น FillMissing ใส่ค่าที่ขาดไป, Categorify สร้าง Embedding สำหรับข้อมูล Category, Normailze สำหรับข้อมูลตัวเลข

*เรื่อง Normalize, Embedding เราจะอธิบายต่อไป*

In [ ]:
dependent_variable = 'salary'
category_column_names = ['workclass', 'education', 
                         'marital-status', 'occupation', 
                         'relationship', 'race']
continuous_column_names = ['age', 'fnlwgt', 'education-num']

preprocesses = [FillMissing, Categorify, Normalize]

แบ่งข้อมูล Record ที่ 800-1000 เอาไว้เป็น Validation Set

In [ ]:
tabularlist_test = TabularList.from_df(dataframe.iloc[800:1000].copy(), 
                           path=path, 
                           cat_names=category_column_names, 
                           cont_names=continuous_column_names, 
                           procs=preprocesses)

สร้าง Databunch จากการกำหนดค่าด้านบน

In [ ]:
databunch = (TabularList.from_df(dataframe, path=path, 
                                 cat_names=category_column_names, 
                                 cont_names=continuous_column_names, 
                                 procs=preprocesses)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dependent_variable)
                           .add_test(tabularlist_test)
                           .databunch())

## สำรวจข้อมูล ##


*เราสามารถรัน cell นี้หลายครั้ง เพื่อเรียกดู batch ต่อ ๆ ไป ได้เรื่อย ๆ เป็นการสำรวจข้อมูล*

In [ ]:
databunch.show_batch(rows=10)

สังเกตตัวเลข อายุมีค่าเป็นติดลบ เนื่องจากการ Normailze จะทำให้ Mean = 0, Standard Deviation = 1 และ มี Column ใหม่ คื education-num_na ขึ้นมาเพื่อบอกว่า มีการเติมค่าที่หายไปให้กับ Record นี้

In [ ]:
databunch.classes

# 4. สร้างโมเดล

เนื่องจากตารางเล็ก ๆ ไม่ซับซ้อน เราจะสร้างโมเดล ที่ประกอบด้วย Dense Layer ที่ Deep แค่ 2 Hidden Layer โดยที่เหลือใช้ค่า Default ทั้งหมด เรื่องชนิดของ Layer เช่น Dense ไว้เราจะอธิบายต่อไป

In [ ]:
learner = tabular_learner(databunch, layers=[256, 128], 
                          metrics=accuracy, 
                          callback_fns=ShowGraph)

# 5. เริ่มต้นเทรนโมเดล

เทรนด้วยค่า Default ไป 1 Cycle สามารถใช้ Learning Rate สูง ๆ ได้เลย

In [ ]:
learner.fit_one_cycle(1, 2e-1)

## สำเร็จแล้ว ##

เพียงแค่เวลา 3 วินาที เราเทรน Model ได้ accuracy ประมาณ 0.84 หรือ ความแม่นยำประมาณ 84%


In [ ]:
learner.save("05a-tabular")

In [ ]:
learner.load("05a-tabular");

# 6. ดูผลลัพธ์

ลองดูข้อมูล Record ที่ 850

In [ ]:
row = dataframe.iloc[850]
row

ให้โมเดลทำนายกัน

In [ ]:
learner.predict(row)


โมเดลทำนายได้ถูกต้อง

# 7. สรุป

1. การเทรนโมเดลหลัก ๆ จะคล้าย ๆ กัน แต่สำหรับข้อมูลแบบตาราง จะมีประเด็นด้านข้อมูลไม่สมบูรณ์ กรอกไม่ครบ ข้อมูลขาดหาย และเนื่องจากโมเดลรับได้แต่ตัวเลข จึงจำเป็นต้องมีการทำ Preprocessing เตรียมข้อมูลก่อนเทรน
2. เราสามารถสร้าง โมเดล Classification จากข้อมูลแบบตารางได้ไม่ยาก ใช้เวลาเทรนไม่นาน ไม่จำเป็นต้องใช้ GPU
3. โมเดลแบบนี้ สามารถนำไปประยุกต์ใช้ในระบบงานออฟฟิศทั่วไป ได้อย่างหลากหลาย เพราะข้อมูลในภาคธุรกิจส่วนใหญ่ก็จะอยู่ในรูปแบบตารางแบบนี้อยู่แล้ว

# Credit 

* [FastAI: Practical Deep Learning for Coders, v3 - Lesson 4](https://course.fast.ai/videos/?lesson=4)
* [Adult Data Set](https://archive.ics.uci.edu/ml/datasets/adult)